In [9]:
#Packages being downloaded
import csv
import pandas as pd
import re
import xml.etree.ElementTree as ET

from tkinter import filedialog
from tkinter import *

import os

In [2]:
def getFile(sdtFile):
    root = Tk()
    root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("text files","*.csv"),("all files","*.*")))
    changePath(root.filename,sdtFile)
    root.destroy()
    return(root.filename)

In [3]:
def changePath(path,sdtFile):
    tree = ET.parse(sdtFile)
    root = tree.getroot()
    
    for elem in root.findall('.//Datasources/Datasource/Host'):
        elem.text=path
    
    for elem in root.findall('.//Datasources/Datasource/Name'):
        elem.text=path
        
    tree.write(sdtFile,encoding="utf-8", xml_declaration=True)

In [4]:
def readCSV(csvName):
    df = pd.read_csv(csvName)
    print("reading csv done")
    return df

In [5]:
def getCat(df,colorName):
    x=0
    for name in df.columns:
        if df.dtypes[x] == 'object':
             if len(df[name].astype('category').cat.categories.tolist()) >1 and len(df[name].astype('category').cat.categories.tolist()) <101 and colorName=="": 
                colorName=df.columns[x]
        x= x+1
    return colorName

In [6]:
def getCatCol(df,colorName):
    return df[colorName].astype('category').cat.categories.tolist()

In [7]:
def typeDefiner(df,catFound,sdtFile):
    colList = df.columns
    colList
    x = 0
    colNames = []
    for name in colList:
        if df.dtypes[x] == 'object':# or df.dtypes[x] == 'int64':
            pass
        else:
            colNames.append(colList[x])
                
        x = x + 1
    
    file = ET.parse(sdtFile)
    root = file.getroot()
    inputCount=0
    for elem in root.findall('.//InputFields'):
        for inputs in elem.findall('.//InputField'):
            inputCount+=1
    
    if not catFound and inputCount>len(colNames):
        resetToDefault(sdtFile)
    print("input count is "+str(catFound)+str(len(colNames)))
    print("objects removed")
    print(colNames)
    return colNames

In [8]:
def toNumeric(sdtFile):

    #import xml.etree.ElementTree as ET
    #CLEARS COLOR OUT NOW WE CAN INSERT ANYTHING
    file = ET.parse(sdtFile)
    root = file.getroot()
    for elem in root.findall('.//Glyph/Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
        elem.clear()
        

    for parent in root.findall('.//KeyValuePairs/..'):
        for element in parent.findall('KeyValuePairs'):
            parent.remove(element)
            
    
   #Try to write here
    for elem in root.findall('.//Glyph/Color/RGB/..'):
        check_req_elems = elem.find('.//Function')
        #check_req_elems.type = "Linear Interpolation"
        check_req_elems.set("type","Linear Interpolation")
        attrib = {'type': 'BoundField'}
        minmax = ET.SubElement(check_req_elems, 'MinMax', attrib)
        minmax.text= "\n\t\t\t\t\t\t\t"
        min = ET.SubElement(minmax, 'Min')
        min.text = '0'
        min.tail="\n\t\t\t\t\t\t"
        minmax.tail="\n\t\t\t\t\t"
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)
    print("color fixed")

In [9]:
def XMLParser(colNames, sdtFile,colorName,catFound):
    print(colNames)

    tree = ET.parse(sdtFile)
    root = tree.getroot()
    count=0;
    print(len(colNames))
    for elem in root.iter('InputField'):
        print(count)
        if elem.get('name')!="color":
            elem.set('field', colNames[count])
            count+=1
        elif not catFound:
            elem.set('field', colNames[count])
            count+=1
        else:
            elem.set('field', colorName)
        
    #tree.write('New DAL 900.sdt')
    tree.write(sdtFile)
    print("XML updated")

In [10]:
def runAll(sdtFile):
    colorCol= None
    colorName=""
    catFound=False
    csv = getFile(sdtFile)
    df = readCSV(csv)
    colorName=getCat(df,colorName)
    print(colorName)
    if(colorName!=""):
        catFound=True
        colorCol=getCatCol(df,colorName)
    
    print(catFound)
    sdtFile=moveToNewFile(sdtFile,csv)
    colNames = typeDefiner(df,catFound,sdtFile)
    #colNames.append(colorName)
    XMLParser(colNames, sdtFile,colorName,catFound)
    setupColor(colorCol, sdtFile)
   # csv = getFile()
#readCSV(csv)

In [15]:
#%%python
runAll('emptySDT.sdt')

reading csv done
Hitter_Team
True
input count is True13
objects removed
['id', 'dateTime', 'Inning', 'True_Distance', 'Speed_off_Bat', 'Elevation_Angle', 'Horizontal_Angle', 'Apex', 'Angle_Adj', 'Location_X', 'Location_Y', 'year', 'month']
['id', 'dateTime', 'Inning', 'True_Distance', 'Speed_off_Bat', 'Elevation_Angle', 'Horizontal_Angle', 'Apex', 'Angle_Adj', 'Location_X', 'Location_Y', 'year', 'month']
13
0
1
2
3
XML updated
Setting up Colors
117,99,87
231,84,13
222,152,157
138,169,250
10,45,79
253,72,75
235,239,194
31,137,126
179,194,80
79,79,241
111,227,239
158,129,221
222,131,192
200,182,186
250,97,3
74,116,146
227,207,192
56,250,17
244,35,91
25,68,106
87,250,190
123,187,198
122,191,142
35,227,175
245,249,95
208,80,12
103,97,193
225,223,15
198,0,152
180,222,186
91,130,46
51,254,131
204,153,117
198,146,38
174,158,68
175,166,133
120,130,55
243,95,194
97,171,209
86,57,177
63,80,155
4,125,206
163,179,170
166,77,29
19,102,89
144,159,103
130,138,128
163,150,139
132,229,179
45,176,115
20

# Extra functions

In [4]:
def elevationFinder(df):
    if df.columns.contains("elevation") or df.columns.contains("Elevation") or df.columns.contains("height"):
        longitude = df["Longitude"]
    else:
        longitude = "none found"
    return longitude

In [3]:
def latFinder2(df):
    x = 0
    for columns in df.columns:
        searchObj = re.match( r'LAT', df.columns[x], re.I)
        if searchObj:
            #print("found it")
            lat = df.columns[x]
            return lat
        else:
            x = x + 1
            pass

In [2]:
def longFinder2(df):
    x = 0
    for columns in df.columns:
        searchObj = re.match( r'LONG', df.columns[x], re.I)
        if searchObj:
            #print("found it")
            long = df.columns[x]
            return long
        else:
            x = x + 1
            pass

In [11]:
def setupColor(colorCol,sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    x=0
    if(colorCol==None):#and then potentially add a binding field
        print("Changing to numeric")
        toNumeric(sdtFile)
        
    else:
        print("Setting up Colors")
        for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
            for element in elem.findall('KeyValuePair/Key'):
                element.text=colorCol[x]
                x=x+1

            if(len(colorCol)>2):
                while x<len(colorCol):
                    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]'):
                        pairs = elem.find('.//KeyValuePairs')
                #fixes spacing so new pair will be at right column/indent area
                        for child in pairs.findall('.//KeyValuePair'):
                            child.tail="\n\t\t\t\t\t\t\t"
                        keyVal =ET.SubElement(pairs, 'KeyValuePair')
                        keyVal.text="\n\t\t\t\t\t\t\t\t"
                        keyVal.tail="\n\t\t\t\t\t\t"
                        keyValP= ET.SubElement(keyVal, 'Key')
                        keyValP.text=colorCol[x]
                        keyValP.tail="\n\t\t\t\t\t\t\t\t"
                        key = ET.SubElement(keyVal, 'Value')
                    
                        key.tail="\n\t\t\t\t\t\t\t"
                        x=x+1
    
                        x=0

                        count=0
                        colorList=[] 
                        while count<len(colorCol):
                            randomCol(colorList)
                            count=count+1


                        for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
                            for element in elem.findall('KeyValuePair/Value'):
                                element.text=colorList[x]
                                print(colorList[x])
                                x=x+1
                
                
            file.write(sdtFile,encoding="utf-8", xml_declaration=True)


In [12]:
#
def moveToNewFile(sdtFile,csv):
    file = ET.parse(sdtFile)
    base=os.path.basename(csv)
    newFile =os.path.splitext(base)[0]+".sdt"
    file.write(newFile,encoding="utf-8", xml_declaration=True)
    
    return newFile

In [13]:
def randomCol(colorList):
    import random
    r = random.random()*255
    r=int(r)
    g = random.random()*255
    g=int(g)
    b = random.random()*255
    b=int(b)
    color=str(r)+","+str(g)+","+str(b)
    colorList.append(color)

In [14]:
def resetToDefault(sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    print("resetting")
    element=None
    for parent in root.findall('.//Glyph/Color/RGB'):
        for e in parent:
            print(e)
            if(element==None):
                element = parent.find('Binding')
                print(element)
        if(element!=None):
            parent.remove(element)
            element=None
    
    for parent in root.findall('.//InputFields'):
        for e in parent:
            print(e)
            if(element==None):
                element =  parent.find('InputField[@name="color"]')
                print(element)
                print("Element found")
        if (element!=None):
            parent.remove(element)
            element=None
    
    count=0
    for parent in root.findall('.//InputFields'):
        for e in parent:
            if(count==len(parent)-1):
                e.tail="\n\t"
            count+=1
        
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)

In [11]:
def MakeDict(sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    for parent in root.findall('.//Glyph'):
        for elem in parent:
            for e in elem:
                if elem.find('Binding'):
                    
    

In [12]:
MakeDict('emptySDT.sdt')

<Element 'Position' at 0x00000081779C51D8>
<Element 'Rotation' at 0x00000081779C5868>
<Element 'Scale' at 0x00000081779C5D68>
<Element 'Color' at 0x00000081779C43B8>
<Element 'Tag' at 0x00000081779C49F8>
<Element 'Description' at 0x00000081779C4AE8>
<Element 'RotationRate' at 0x00000081779C4BD8>
<Element 'Geometry' at 0x00000081779C9138>
<Element 'VirtualTopology' at 0x00000081779C9318>
<Element 'URL' at 0x00000081779C9458>
<Element 'Children' at 0x00000081779C9548>
<Element 'Position' at 0x00000081779C95E8>
<Element 'Rotation' at 0x00000081779C9AE8>
<Element 'Scale' at 0x00000081779C1048>
<Element 'Color' at 0x00000081779C1638>
<Element 'Tag' at 0x00000081779C1A48>
<Element 'Description' at 0x00000081779C1B38>
<Element 'RotationRate' at 0x00000081779C1C28>
<Element 'Geometry' at 0x00000081779D4188>
<Element 'VirtualTopology' at 0x00000081779D4368>
<Element 'URL' at 0x00000081779D44A8>
